In [2]:
import pandas as pd
import requests
import os
from io import StringIO
import numpy as np

# Getting FX Data

In [3]:
def _pull_data(ccy):
    print(ccy)
    # Making an API request to AlphaVantage for FX daily data
    response = requests.get(f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol={ccy}&outputsize=full&datatype=csv&apikey=YOUR_API_KEY')
    # Use StringIO to simulate a file for pandas read_csv
    df = pd.read_csv(StringIO(response.text))
    df['Date'] = pd.to_datetime(df['timestamp'])
    df['c'] = df['close'].astype(float)
    df['ccy'] = ccy
    df = df.sort_values('Date')
    df['LogReturn'] = df['c'].apply(lambda x: np.log(x)).diff()
    return df

def pull_data(ccy):
    if os.path.isfile(f"{ccy}.csv"):
        df = pd.read_csv(f"{ccy}.csv")
    else:
        df = _pull_data(ccy)
        df.to_csv(f"{ccy}.csv", index=False)
    return df

In [4]:
ccys = ["JPY", "CNH", "SGD", "THB", "HKD", "KRW", "TWD"]
results = pd.concat([pull_data(ccy) for ccy in ccys])

results = results.sort_values('Date')
results['LogReturn'] = results.groupby('ccy')['c'].transform(lambda x: np.log(x).diff())
results = results[results['Date'] >= pd.Timestamp("2014-11-24")]

JPY
CNH
SGD
THB
HKD
KRW
TWD
